In [1]:
!pip install requests
!pip install pandas
!pip install time
!pip install ssl
!pip install bs4
!pip install openpyxl
!pip install numpy pandas matplotlib
!pip install nltk
!pip install pymystem3
!pip install selenium
!pip install webdriver-manager
!pip install wordcloud
import requests
import json
import unicodedata
from bs4 import BeautifulSoup
import pandas as pd
import ssl
import time as t
import re

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


  Using cached ssl-1.16.tar.gz (33 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [28 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 14, in <module>
    File "D:\Anaconda\Lib\site-packages\setuptools\__init__.py", line 266, in <module>
      monkey.patch_all()
    File "D:\Anaconda\Lib\site-packages\setuptools\monkey.py", line 98, in patch_all
      patch_for_msvc_specialized_compiler()
    File "D:\Anaconda\Lib\site-packages\setuptools\monkey.py", line 165, in patch_for_msvc_specialized_compiler
      patch_func(*msvc14('_get_vc_env'))
                  ^^^^^^^^^^^^^^^^^^^^^
    File "D:\Anaconda\Lib\site-packages\setuptools\monkey.py", line 155, in patch_params
      mod = import_module(mod_name)
            ^^^^^^^^^^^^^^^^^^^^^^^
    File "D:\Anaconda\Lib\importlib\__init__.py", line 126, in import_module
      return _bootstrap._gcd_import(name[level:], package,

## Parser for Banki.ru

Если мы используем этот код в "производстве", мы хотим постоянно обновлять базу отзывов. Так как отзывы на банк постоянно прибавляются, мы хотим автоматически отследить, сколько страниц содержат отзывы на Тиньков Банк. Следующий код отправляет GET запросы на URL, и получая положительный ответ (200), считает ссылку валидной. Ссылка, которая не отвечает кодом 200 на запрос, является несуществующей, и таким образом, предыдущая перед недействительной ссылка является последней. Если код используется "в производстве", лучше инициировать счетчик ссылок с "предыдущей" последней страницы для экономии времени.

In [2]:
import requests

def send_get_and_get_200(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            print(f"GET request to {url} successful (Status Code: 200)")
            return url
        else:
            print(f"GET request to {url} failed (Status Code: {response.status_code})")
            return None
    except requests.exceptions.RequestException as e:
        print("An error occurred:", e)
        return None

#base url
base_url = "https://www.banki.ru/services/responses/bank/tcs/?page="

# variable to store last URL to define a number we will go to
last_successful_url = None

# starting from the page 2, it is where we get a countetr
page_number = 2

# looping until last request fails
while True:
    url = base_url + str(page_number) + "&is_countable=on"
    # sending the request to connect
    result = send_get_and_get_200(url)
    # update the last request
    if result:
        last_successful_url = result
    else:
        # if the request fails, break the loop
        break
    if page_number==15:
      break
    # increment to iterateh
    page_number += 1

# last successful url for us to know
if last_successful_url:
    print("Last successful URL:", last_successful_url)
else:
    print("No successful URL found.")


GET request to https://www.banki.ru/services/responses/bank/tcs/?page=2&is_countable=on successful (Status Code: 200)
GET request to https://www.banki.ru/services/responses/bank/tcs/?page=3&is_countable=on successful (Status Code: 200)
GET request to https://www.banki.ru/services/responses/bank/tcs/?page=4&is_countable=on successful (Status Code: 200)
GET request to https://www.banki.ru/services/responses/bank/tcs/?page=5&is_countable=on successful (Status Code: 200)
GET request to https://www.banki.ru/services/responses/bank/tcs/?page=6&is_countable=on successful (Status Code: 200)
GET request to https://www.banki.ru/services/responses/bank/tcs/?page=7&is_countable=on successful (Status Code: 200)
GET request to https://www.banki.ru/services/responses/bank/tcs/?page=8&is_countable=on successful (Status Code: 200)
GET request to https://www.banki.ru/services/responses/bank/tcs/?page=9&is_countable=on successful (Status Code: 200)
GET request to https://www.banki.ru/services/responses/b

Ради демонстрации решения мы берем первые 10 страниц, пытается собрать все ссылки, которые содержат URLs. Далее, собранные ссылки фильтруются для выделения уникальных айди отзывов.

In [ ]:
url = "https://www.banki.ru/services/responses/bank/tcs/?page=2&is_countable=on"

from urllib.parse import urlparse
def find_urls(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            links = soup.find_all('a', href=True)
            urls = [link['href'] for link in links]
            urls = [u if urlparse(u).scheme in ['http', 'https'] else url + u for u in urls]
            extracted_numbers = [re.search(r'/responses/bank/response/(\d+)/', u).group(1) for u in urls if re.search(r'/responses/bank/response/(\d+)/', u)]
            unique_extracted_numbers = list(set(extracted_numbers))
            return unique_extracted_numbers, urls
            return urls
        else:
            print("Failed to fetch the webpage. Status code:", response.status_code)
            return []
    except Exception as e:
        print("An error occurred:", e)
        return []

base_url = "https://www.banki.ru/services/responses/bank/tcs/?page="
all_extracted_numbers = []

for page_number in range(1, 10):
    url = base_url + str(page_number) + "&is_countable=on"
    extracted_numbers, _ = find_urls(url)
    all_extracted_numbers.extend(extracted_numbers)

len(all_extracted_numbers)
unique_extracted_numbers = list(set(all_extracted_numbers))

In [4]:

def clean_html(html_content):
    cleaned_content = ''.join(char for char in html_content if unicodedata.category(char)[0] != 'C')
    return cleaned_content

def extract_review_info(url):
    try:
        # Fetch the HTML content from the URL
        response = requests.get(url)
        if response.status_code == 200:
            html_content = response.text.replace('\x00', '')
            soup = BeautifulSoup(html_content, 'html.parser')
            script_tags = soup.find_all('script', type='application/ld+json')
            for script_tag in script_tags:
                script_content = script_tag.contents[0].strip()  # Get the content of the script tag
                script_data = json.loads(script_content)  # Parse JSON data
                if script_data.get('@type', '') == 'Review':
                    review_info = {
                        'review_name': script_data.get('name', ''),
                        'author_name': script_data.get('author', {}).get('name', ''),
                        'rating_value': script_data.get('reviewRating', {}).get('ratingValue', ''),
                        'review_body': script_data.get('author', {}).get('reviewBody', '')
                    }
                    return review_info
            # If review info is not found
            return None
        else:
            print(f"Failed to fetch the webpage at {url}. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred while fetching or parsing the HTML content: {e}")
        return None



#try with this url
url = "https://www.banki.ru/services/responses/bank/response/11397996/"

review_body = extract_review_info(url)

print(review_body)


{'review_name': 'Ложный запрос на кредитную карту', 'author_name': 'gargantua', 'rating_value': '1', 'review_body': '&lt;p&gt;Позвонили сегодня из банка, предложили кредитку. Я отказался. Зайдя через некоторое время в приложение обнаружил, что я, оказывается, сделал заявку, и ее отклонили. Это уже третий случай заявки на кредит от моего имени с последующим отказом, о которых я не подозревал, с декабря месяца, результате чего мой кредитный рейтинг обрушился на 300 пунктов и продолжает падать. Поддержка каждый раз извиняется и обещает ситуацию исправить, но вместо этого ее усугубляет.\xa0&lt;/p&gt;'}


In [5]:
df_reviews = pd.DataFrame()
url_template = "https://www.banki.ru/services/responses/bank/response/{}/"
for number in all_extracted_numbers[:10]:
    url = url_template.format(number)
    review_info = extract_review_info(url)
    if review_info:
      df_reviews = pd.concat([df_reviews, pd.DataFrame({'review_name': [review_info['review_name']],
                                                  'author_name': [review_info['author_name']],
                                                  'rating_value': [review_info['rating_value']],
                                                  'review_body': [review_info['review_body']]})])

An error occurred while fetching or parsing the HTML content: Invalid control character at: line 18 column 259 (char 675)
An error occurred while fetching or parsing the HTML content: Invalid control character at: line 18 column 205 (char 571)
An error occurred while fetching or parsing the HTML content: Invalid control character at: line 18 column 508 (char 888)
An error occurred while fetching or parsing the HTML content: Invalid control character at: line 18 column 412 (char 777)
An error occurred while fetching or parsing the HTML content: Invalid control character at: line 18 column 271 (char 639)
An error occurred while fetching or parsing the HTML content: Invalid control character at: line 18 column 720 (char 1104)
An error occurred while fetching or parsing the HTML content: Invalid control character at: line 18 column 397 (char 754)


In [6]:
df_reviews['source'] = 'banki.ru'

In [7]:
df_reviews

,review_name,author_name,rating_value,review_body,source
0,Быстро решили проблему с кэшбеком,user-126513506494,5,&lt;p&gt;22.03.2024 Оплатил услуги автосервиса...,banki.ru
0,Оспаривание операции,Didi1989i,5,&lt;p&gt;Заказала через приложение «Магнит» до...,banki.ru
0,Лучший сотрудник месяца Дарья,user-228817535647,5,Сотрудник Дарья помогла разобраться с непонятн...,banki.ru


## Parser for Sravni.ru


In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
import pandas as pd

# Константа, определяющая, сколько отзывов нужно загрузить
NUM_REVIEWS_TO_LOAD = 10

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
wait = WebDriverWait(driver, 20)

driver.get("https://www.sravni.ru/bank/tinkoff-bank/otzyvy/")

review_data = []

try:
    loaded_reviews = 0
    while loaded_reviews < NUM_REVIEWS_TO_LOAD:
        # Ждем появления отзывов на странице
        reviews = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".review-card_wrapper__gnPSK")))

        # Определяем последний отзыв на странице
        last_review = reviews[-1]
        driver.execute_script("arguments[0].scrollIntoView(true);", last_review)
        time.sleep(1)  # Небольшая задержка для стабильности

        for review in reviews[loaded_reviews:]:
            try:
                # Пытаемся найти и кликнуть по кнопке "Читать"
                read_more_button = review.find_element(By.CSS_SELECTOR, "a.h-ml-4._p7lcln._7k5cz5")
                driver.execute_script("arguments[0].scrollIntoView(true);", read_more_button)
                time.sleep(1)  # Небольшая задержка для стабильности
                read_more_button.click()
                time.sleep(1)  # Ожидаем раскрытия отзыва
            except Exception as e:
                print("Кнопка 'Читать' не найдена или другая ошибка:", e)

            # Извлекаем данные отзыва
            try:
                review_name = review.find_element(By.CSS_SELECTOR, ".review-card_title__zYdxx").text
                author_name = review.find_element(By.CSS_SELECTOR, ".h-color-D100._1h41p0x._1livb46._1gpt55s").text
                rating_elements = review.find_elements(By.CSS_SELECTOR, "._e7kry4._akgnn3 span[data-qa='Icon']")
                date = review.find_element(By.CSS_SELECTOR, ".h-color-D30._1h41p0x").text
                rating_value = len(rating_elements)
                review_text_element = review.find_element(By.CSS_SELECTOR, ".review-card_text__jTUSq")
                review_text = review_text_element.text.replace("\nЧитать", "").rstrip()  # Удаляем "Скрыть", если присутствует

                if review_text.endswith("Скрыть"):
                    review_text = review_text[:-6]

                review_info = {
                    "review_name":review_name,
                    "author_name": author_name,
                    "date": date,
                    "rating_value": rating_value,
                    "review_body": review_text
                }

                print(review_info)
                review_data.append(review_info)
                loaded_reviews += 1

                if loaded_reviews >= NUM_REVIEWS_TO_LOAD:
                    break
            except Exception as e:
                print("Ошибка при извлечении данных отзыва:", e)

finally:
    with open("sravni.json", "w", encoding="utf-8") as f:
        json.dump(review_data, f, ensure_ascii=False, indent=4)

    driver.quit()

# Конвертация данных в DataFrame
df_reviews_selenium = pd.DataFrame(review_data)


Кнопка 'Читать' не найдена или другая ошибка: Message: element click intercepted: Element <a class="h-ml-4 _p7lcln _7k5cz5">...</a> is not clickable at point (794, 11). Other element would receive the click: <div class="side-block_buttonWrapper__6q8p1">...</div>
  (Session info: chrome=123.0.6312.86)
Stacktrace:
	GetHandleVerifier [0x00A04CC3+225091]
	(No symbol) [0x00934E11]
	(No symbol) [0x007D9A7A]
	(No symbol) [0x00816D42]
	(No symbol) [0x00815574]
	(No symbol) [0x008135DB]
	(No symbol) [0x008127BA]
	(No symbol) [0x008098DE]
	(No symbol) [0x0082F55C]
	(No symbol) [0x0080930E]
	(No symbol) [0x0082F7F4]
	(No symbol) [0x00845CB0]
	(No symbol) [0x0082F2F6]
	(No symbol) [0x008079B9]
	(No symbol) [0x0080879D]
	sqlite3_dbdata_init [0x00E79A63+4064547]
	sqlite3_dbdata_init [0x00E8106A+4094762]
	sqlite3_dbdata_init [0x00E7B968+4072488]
	sqlite3_dbdata_init [0x00B7C9C9+930953]
	(No symbol) [0x009407E4]
	(No symbol) [0x0093AD08]
	(No symbol) [0x0093AE31]
	(No symbol) [0x0092CAA0]
	BaseThreadI

In [ ]:
df_reviews_selenium['source']='sravni'
df_reviews_selenium


# Merging and Cleaning

In [10]:
merged_raw.shape

NameError: name 'merged_raw' is not defined

In [ ]:
merged_raw = pd.concat([df_reviews, df_reviews_selenium], ignore_index=True)
merged_raw

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from pymystem3 import Mystem

class Text_Cleaner:
    def __init__(self, df, column, numeric_column):
        self.df = df
        self.column = column
        self.numeric_column = numeric_column
        nltk.download('stopwords')
        self.stop_words = set(stopwords.words('russian'))
        self.stemmer = SnowballStemmer('russian')
        self.mystem = Mystem()

    def remove_stopwords(self, text):
        """
        Removes Russian stop words from the text.
        """
        cleaned_text = ' '.join([word for word in text.split() if word.lower() not in self.stop_words])
        return cleaned_text

    def remove_emojis(self, text):
        """
        Removes emojis from the text.
        """
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002500-\U00002BEF"  # chinese char
                                   u"\U00002702-\U000027B0"
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001f926-\U0001f937"
                                   u"\U00010000-\U0010ffff"
                                   u"\u2640-\u2642"
                                   u"\u2600-\u2B55"
                                   u"\u200d"
                                   u"\u23cf"
                                   u"\u23e9"
                                   u"\u231a"
                                   u"\ufe0f"  # dingbats
                                   u"\u3030"
                                   "]+", flags=re.UNICODE)
        cleaned_text = emoji_pattern.sub(r'', text)
        return cleaned_text

    def text_cleaner(self, text):
        """
        Cleans the text by removing HTML tags, stopwords, emojis, HTML entities, and performs additional cleaning.
        """
        if pd.isna(text):
            return ''
        # Remove HTML tags and entities
        cleaned_text = re.sub(r'<[^>]+>|&lt;|&gt;', '', text)
        # Remove stopwords and emojis
        cleaned_text = self.remove_stopwords(cleaned_text)
        cleaned_text = self.remove_emojis(cleaned_text)
        # Perform additional cleaning
        cleaned_text = cleaned_text.replace('&quot;', '').replace('&amp;', '').replace('&lt;', '').replace('&gt;', '')
        return cleaned_text

    def lemmatize_text(self, text):
        """
        Lemmatizes the text using Mystem.
        """
        lemmatized_words = self.mystem.lemmatize(text.lower())
        return ' '.join(lemmatized_words)

    def stem_text(self, text):
        """
        Stems the text using SnowballStemmer.
        """
        stemmed_text = ' '.join([self.stemmer.stem(word) for word in text.split()])
        return stemmed_text

    def clean_text_column(self):
        """
        Applies text cleaning functions to the specified column of the DataFrame.
        """
        self.df.dropna(inplace=True)  # Drop NaN values
        # Clean the text column
        cleaned_column = self.df[self.column].apply(self.text_cleaner)
        self.df[self.column + '_cleaned'] = cleaned_column
        # Lemmatize the cleaned text
        lemmatized_column = cleaned_column.apply(self.lemmatize_text)
        self.df[self.column + '_lemmatized'] = lemmatized_column
        # Stem the cleaned text
        stemmed_column = cleaned_column.apply(self.stem_text)
        self.df[self.column + '_stemmed'] = stemmed_column
        # Convert numeric column to numeric type
        self.df[self.numeric_column] = pd.to_numeric(self.df[self.numeric_column], errors='coerce')

    def cluster_reviews(self, num_clusters=5):
        """
        Placeholder method for clustering reviews.
        """
        # This method can be implemented to perform clustering if needed
        pass

# Example usage:
# Assuming df_reviews is your DataFrame and 'review_body' is the column containing text data,
# 'rating_value' is the column containing numeric ratings.

# Create an instance of Text_Cleaner
cleaner = Text_Cleaner(df_reviews, 'review_body', numeric_column='rating_value')

cleaner.clean_text_column()


In [ ]:
df_reviews

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from pymystem3 import Mystem

class Text_Cleaner:
    def __init__(self, df, column, numeric_column):
        self.df = df
        self.column = column
        self.numeric_column = numeric_column
        nltk.download('stopwords')
        self.stop_words = set(stopwords.words('russian'))
        self.stemmer = SnowballStemmer('russian')
        self.mystem = Mystem()

    def remove_stopwords(self, text):

        cleaned_text = ' '.join([word for word in text.split() if word.lower() not in self.stop_words])
        return cleaned_text

    def remove_emojis(self, text):

        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  
                                   u"\U0001F300-\U0001F5FF"  
                                   u"\U0001F680-\U0001F6FF"  
                                   u"\U0001F1E0-\U0001F1FF"  
                                   u"\U00002500-\U00002BEF"  
                                   u"\U00002702-\U000027B0"
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001f926-\U0001f937"
                                   u"\U00010000-\U0010ffff"
                                   u"\u2640-\u2642"
                                   u"\u2600-\u2B55"
                                   u"\u200d"
                                   u"\u23cf"
                                   u"\u23e9"
                                   u"\u231a"
                                   u"\ufe0f"  # dingbats
                                   u"\u3030"
                                   "]+", flags=re.UNICODE)
        cleaned_text = emoji_pattern.sub(r'', text)
        return cleaned_text

    def text_cleaner(self, text):
        if pd.isna(text):
            return ''
        
        cleaned_text = re.sub(r'<[^>]+>|&lt;|&gt;', '', text)
        cleaned_text = cleaned_text.replace('&lt;p&gt;', '')
        cleaned_text = cleaned_text.lower()
        cleaned_text = self.remove_stopwords(cleaned_text)
        cleaned_text = self.remove_emojis(cleaned_text)
        cleaned_text = cleaned_text.replace('&quot;', '').replace('&amp;', '').replace('&lt;', '').replace('&gt;', '')
        return cleaned_text

    def lemmatize_text(self, text):
        lemmatized_words = self.mystem.lemmatize(text.lower())
        return ' '.join(lemmatized_words)

    def stem_text(self, text):
        tokens = text.split()  # Tokenize the text
        stemmed_tokens = [self.stemmer.stem(token) for token in tokens]
        return ' '.join(stemmed_tokens)

    def clean_text_column(self):
        self.df[self.column] = self.df[self.column].apply(self.text_cleaner)

    def cluster_reviews(self, num_clusters=5):
        pass

cleaner = Text_Cleaner(merged_raw, 'review_body', numeric_column='rating_value')
cleaner.clean_text_column()


In [ ]:
merged_raw